In [36]:
import os
from PIL import Image
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F


# Define directories
train_directory = './Bone_Fracture_Binary_Classification/train/'
test_directory = './Bone_Fracture_Binary_Classification/test/'
valid_directory = './Bone_Fracture_Binary_Classification/val/'

classes = ['Fractured', 'non- Fractured']

# Define transformations
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Normalizing with ImageNet mean and std
    ]),

    'test': transforms.Compose([
        transforms.Resize((224, 32242)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),

    'valid': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'singleImage': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])

    ])

}

# Create datasets
train_dataset= datasets.ImageFolder(train_directory, transform=data_transforms['train'])
test_dataset = datasets.ImageFolder(test_directory, transform=data_transforms['test'])
valid_dataset= datasets.ImageFolder(valid_directory, transform=data_transforms['valid'])
# singleImage_dataset= datasets.ImageFolder(valid_directory, transform=data_transforms['singleImage'])

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False)

# device 
device = 'cuda' if torch.cuda.is_available else 'cpu'


In [37]:
iterator = iter(train_loader)
print(next(iterator))

[tensor([[[[-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          ...,
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179]],

         [[-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          ...,
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357]],

         [[-1.8044, -1.8044, -1.8044,  ..., -1.8044, -1.8044, -1.8044],
          [-1.8044, -1.8044, 

In [43]:
class XrayClassifierModel(nn.Module):
    def __init__(self) -> None:
        super(XrayClassifierModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 24, 5)
        self.pool = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(24,46,5)
        self.conv3 = nn.Conv2d(46,92,5)
        self.conv4 = nn.Conv2d(92,120,5)
        self.fc1 = nn.Linear(120*10*10, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84,2)
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = self.pool(F.relu(self.conv4(x)))
        x = x.view(-1, 120*10*10)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        # x = F.sigmoid(x)
        x = self.fc3(x)
        return x
        


In [44]:
def train(dataloader, model, criterion, optimizer, num_epochs):
    num_epoch = num_epochs
    learning_rate = 0.01
    model = model
    for x in range(num_epoch):
        for i, (images, labeles) in enumerate(train_loader):
            images, labeles = images.to(device), labeles.to(device)
            pred = model(images)
            loss = criterion(pred, labeles)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            if(i % 100 == 0):
                print(f'Epochs: {num_epoch}/{x+1} loss: {loss.item():.4f}')
    print('The training Finished')
    
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [47]:
num_epoch = 8
learning_rate = 0.01
model = XrayClassifierModel().to(device)
criterion = nn.CrossEntropyLoss()
optimizer= torch.optim.SGD(model.parameters(), lr=learning_rate)

# Train the model
train(model=model,dataloader=train_loader, criterion=criterion, optimizer=optimizer, num_epochs=num_epoch)
torch.save(model.state_dict(), "./model/model.pth")

Epochs: 8/1 loss: 0.6940
Epochs: 8/1 loss: 0.6921
Epochs: 8/1 loss: 0.6913
Epochs: 8/2 loss: 0.6898
Epochs: 8/2 loss: 0.6861
Epochs: 8/2 loss: 0.6921
Epochs: 8/3 loss: 0.6369
Epochs: 8/3 loss: 0.6837
Epochs: 8/3 loss: 0.7051
Epochs: 8/4 loss: 0.5785
Epochs: 8/4 loss: 0.6019
Epochs: 8/4 loss: 0.5289
Epochs: 8/5 loss: 0.4604
Epochs: 8/5 loss: 0.3908
Epochs: 8/5 loss: 0.1690
Epochs: 8/6 loss: 0.2152
Epochs: 8/6 loss: 0.1833
Epochs: 8/6 loss: 0.1003
Epochs: 8/7 loss: 0.0373
Epochs: 8/7 loss: 0.0708
Epochs: 8/7 loss: 0.0128
Epochs: 8/8 loss: 0.0104
Epochs: 8/8 loss: 0.0168
Epochs: 8/8 loss: 0.0100
The training Finished


In [ ]:
# Testing 
model = XrayClassifierModel()
model.load_state_dict(torch.load('./model/model.pth'))
model.to(device=device)
test(dataloader=test_loader, model=model, loss_fn=criterion)